In [1]:
import pyscf
import ctypes
import numpy as np
from pyscf import dft
from pyscf import gto
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import lib
from pyscf.dft import libxc
from pyscf.dft import numint
from pyscf.dft import xc_deriv

In [2]:
mol=gto.Mole()
mol.atom='''
    H 0 0 0
'''#Z-matrix input format
mol.spin=1
mol.basis='sto-3g'
mol.symmetry = True
mol.build()
mylda = dft.UKS(mol)
mylda.xc = 'lda,vwn'
mylda.kernel()

converged SCF energy = -0.43567023297957  <S^2> = 0.75  2S+1 = 2


-0.43567023297957

In [3]:
mol=gto.Mole()
mol.atom='''
    H 0 0 0
'''#Z-matrix input format
mol.spin=1
mol.basis='6-31g'
mol.symmetry = True
mol.build()
mylda = dft.UKS(mol)
mylda.xc = 'lda,vwn'
mylda.kernel()

converged SCF energy = -0.476044470492464  <S^2> = 0.75  2S+1 = 2


-0.47604447049246384

In [4]:
mol=gto.Mole()
mol.atom='''
    H 0 0 0
'''#Z-matrix input format
mol.spin=1
mol.basis='cc-pvtz'
mol.symmetry = True
mol.build()
mylda = dft.UKS(mol)
mylda.xc = 'lda,vwn'
mylda.kernel()

converged SCF energy = -0.478347509256095  <S^2> = 0.75  2S+1 = 2


-0.4783475092560955

In [5]:
mol=gto.Mole()
mol.atom='''
    H 0 0 0
'''#Z-matrix input format
mol.spin=1
mol.basis='cc-pvtz'
mol.symmetry = True
mol.build()
mylda = dft.UKS(mol)
#check
mylda.xc = 'lda,vwn'

mylda.kernel()


converged SCF energy = -0.478347509256095  <S^2> = 0.75  2S+1 = 2


-0.4783475092560953

In [6]:
mol=gto.Mole()
mol.atom='''
    H 0 0 0
'''#Z-matrix input format
mol.spin=1
mol.basis='cc-pvtz'
mol.symmetry = True
mol.build()
mylda = dft.UKS(mol)
#check
mylda.xc = 'lda'
mylda.kernel()
print(f"lda type:{dft.xcfun.xc_type('lda')},parse:{dft.xcfun.parse_xc('lda,lda')}")

print(f"lda,vwn type:{dft.xcfun.xc_type('lda,vwn')},parse:;p0{dft.xcfun.parse_xc('lda,vwn')}")

converged SCF energy = -0.456680331798621  <S^2> = 0.75  2S+1 = 2
lda type:LDA,parse:((0, 0, 0), ((0, 2),))
lda,vwn type:LDA,parse:;p0((0, 0, 0), ((0, 1), (3, 1)))


XC_CODES = {
'SLATERX'       :  0,  #Slater LDA exchange
'PW86X'         :  1,  #PW86 exchange
'VWN3C'         :  2,  #VWN3 LDA Correlation functional
'VWN5C'         :  3,  #VWN5 LDA Correlation funct

In [7]:
# get_veff calculates columb energy (J) + XC, and each part is accessbile with .ecoul or .exc
# Coulomb energy (Hartree energy)
print(f"Coulomb energy: {mylda.get_veff().ecoul} Hartree")
# Exchange-correlation energy
print(f"Exchange-correlation energy: {mylda.get_veff().exc} Hartree")

# Nuclear repulsion energy
print(f"Nuclear repulsion energy: {mylda.energy_nuc()} Hartree")

# note we can also access the electronic energy with energy_elec(), which returns: (electronic energy, 2electron contribution)
print(f"Electronic energy: {mylda.energy_elec()[0]} Hartree")

Coulomb energy: 0.2964314063569692 Hartree
Exchange-correlation energy: -0.2546398297770955 Hartree
Nuclear repulsion energy: 0 Hartree
Electronic energy: -0.4566803317986212 Hartree


In [8]:
#default integration grids use Bragg radii for atoms
#Treutler-Ahlrichs radial grids, Becke partitioning for grid weights
grids = dft.gen_grid.Grids(mol)
grids.build(with_non0tab=True)
weights = grids.weights
coords = grids.coords
ao_value_grid = numint.eval_ao(mol, coords, deriv=1)
print(ao_value_grid.shape)
print("dimension for phi matrix:",ao_value_grid[0].shape)
#the value for phi matrix and the first derivative
phi=ao_value_grid[0]
phix=ao_value_grid[1]
phiy=ao_value_grid[2]
phiz=ao_value_grid[3]
dm = mylda.make_rdm1()
rho_grid=  numint.eval_rho(mol, ao_value_grid,dm ,xctype='GGA')
print(rho_grid.shape)

(4, 10024, 14)
dimension for phi matrix: (10024, 14)
(4, 10024)


In [9]:

_itrf= lib.load_library('libxc_itrf')
def evalxc(rho):
    #create a list for fn_ids and facs
    hyb, fn_facs = dft.libxc.parse_xc('lda,vwn')
    fn_ids = [x[0] for x in fn_facs]
    facs   = [x[1] for x in fn_facs]
    omega=[0] * len(facs)
    #XC_NVAR('LDA', 1): (1, 2)
    #only density is the variable
    spin=1
    nvar, xlen = xc_deriv._XC_NVAR['LDA', spin]
    ngrids = rho.shape[-1]
    #deriv=1 (corrlation)
    deriv=1
    #outlen=3
    outlen = lib.comb(xlen+deriv, deriv)
    out = np.zeros((outlen,ngrids))
    n = len(fn_ids)
    density_threshold=0
    _itrf.LIBXC_eval_xc(ctypes.c_int(n),
                            (ctypes.c_int*n)(*fn_ids),
                            (ctypes.c_double*n)(*facs),
                            (ctypes.c_double*n)(*omega),
                            ctypes.c_int(spin), ctypes.c_int(deriv),
                            ctypes.c_int(nvar), ctypes.c_int(ngrids),
                            ctypes.c_int(outlen),
                            rho.ctypes.data_as(ctypes.c_void_p),
                            out.ctypes.data_as(ctypes.c_void_p),
                            ctypes.c_double(density_threshold))
    Exc= np.einsum('p,p,p->',out[0],rho,weights)
    Vxca = np.einsum('pb,p,p,pa->ab', phi, out[1], weights, phi, optimize=True)
    Vxcb = np.einsum('pb,p,p,pa->ab', phi, out[2], weights, phi, optimize=True)
    return Exc, Vxca, Vxcb

In [10]:
# ==> Define function to diagonalize F <==
def diag_F(F, norb):
    F_p = A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :norb]
    D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    return (C, D)

In [11]:
# ==> Set default program options <==
# Maximum SCF iterations
MAXITER = 60
# Energy convergence criterion
E_conv = 1.0e-6


In [12]:
S=mol.intor('int1e_ovlp')
nbf = S.shape[0]
nalpha=mol.nelec[0]
nbeta=mol.nelec[1]
#ndocc=min(nalpha,nbeta)
print('Number of basis functions: %d' % (nbf))
print('Number of singly occupied orbitals: %d' % (abs(nalpha - nbeta)))
print(nalpha)
print(nbeta)
#print('Number of doubly occupied orbitals: %d' % (ndocc))


Number of basis functions: 14
Number of singly occupied orbitals: 1
1
0


In [13]:
#Build ERI Tensor
I = mol.intor('int2e')
print(np.shape(I))
#Build core Hamiltonian
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H = T + V

(14, 14, 14, 14)


In [14]:
#symmetric orthogonalization
from scipy.linalg import fractional_matrix_power
A = fractional_matrix_power(S, -0.5)
# ==> Build alpha & beta CORE guess <==
Ca, Da = diag_F(H, nalpha)
Cb, Db = diag_F(H, nbeta)

# Get nuclear repulsion energy
E_nuc = mol.energy_nuc()
# ==> Pre-Iteration Setup <==
# SCF & Previous Energy
SCF_E = 0.0
E_old = 0.0

In [15]:
# Trial & Residual Vector Lists -- one each for alpha & beta
F_list_a = []
F_list_b = []


# ==> UHF-SCF Iterations <==
print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER+1):
    # Build Fa & Fb matrices
    Ja = np.einsum('pqrs,rs->pq', I, Da, optimize=True)
    Jb = np.einsum('pqrs,rs->pq', I, Db, optimize=True)
    rho_grid = np.einsum('pm,mn,pn->p', phi, Da+Db, phi, optimize=True)

    Exc, Vxca, Vxcb = evalxc(rho_grid)
    Fa = H + (Ja + Jb) +Vxca
    Fb = H + (Ja + Jb) +Vxcb
    
    
    # Append trial vectors to lists
    F_list_a.append(Fa)
    F_list_b.append(Fb)
    
    
    # Compute UHF Energy
    SCF_E = np.einsum('pq,pq->', (Da + Db), H, optimize=True)
    SCF_E += 0.5*np.einsum('pq,pq->', Da, Ja, optimize=True)
    SCF_E += 0.5*np.einsum('pq,pq->', Db, Jb, optimize=True)
    SCF_E += E_nuc
    SCF_E += Exc
    
    
    dE = SCF_E - E_old
   
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E' % (scf_iter, SCF_E, dE))
    
    # Convergence Check
    if (abs(dE) < E_conv):
        break
    E_old = SCF_E
   
    # Compute new orbital guess
    Ca, Da = diag_F(Fa, nalpha)
    Cb, Db = diag_F(Fb, nbeta)
# Post iterations
print('\nSCF converged.')
print('Final UHF Energy: %.8f [Eh]' % SCF_E)
print('')

==> Starting SCF Iterations <==

SCF Iteration   1: Energy = -0.4774698029914032 dE = -4.77470E-01
SCF Iteration   2: Energy = -0.4783840675731585 dE = -9.14265E-04
SCF Iteration   3: Energy = -0.4783474656480308 dE =  3.66019E-05
SCF Iteration   4: Energy = -0.4783782406118109 dE = -3.07750E-05
SCF Iteration   5: Energy = -0.4783474920797357 dE =  3.07485E-05
SCF Iteration   6: Energy = -0.4785629999754800 dE = -2.15508E-04
SCF Iteration   7: Energy = -0.4783472917463362 dE =  2.15708E-04
SCF Iteration   8: Energy = -0.4783925627632760 dE = -4.52710E-05
SCF Iteration   9: Energy = -0.4783474777926708 dE =  4.50850E-05
SCF Iteration  10: Energy = -0.4783537573180117 dE = -6.27953E-06
SCF Iteration  11: Energy = -0.4783475081514291 dE =  6.24917E-06
SCF Iteration  12: Energy = -0.4783738457725520 dE = -2.63376E-05
SCF Iteration  13: Energy = -0.4783474961518179 dE =  2.63496E-05
SCF Iteration  14: Energy = -0.4785629882161940 dE = -2.15492E-04
SCF Iteration  15: Energy = -0.478347291910

converged SCF energy = -0.4783475092560953  <S^2> = 0.75  2S+1 = 2
lda type:LDA,parse:((0, 0, 0), ((0, 2),))
lda,vwn type:LDA,parse:;p0((0, 0, 0), ((0, 1), (3, 1)))
